In [5]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import numpy as np
import time

In [3]:
pokemon = pd.read_csv("../pokemon_moves.csv", index_col=0)

In [4]:
links = pd.read_csv("links_moves.csv", index_col=0)

In [10]:
headers = {"User-Agent": "anurooopadevds@gmail.com"}

In [31]:
# Creating a function for finding stats of a pokemon 
def scrap_pokemon_moves(link, headers):
    time.sleep(0.1)
    try:
        #print(f"https://www.serebii.net{link}/")
        response = requests.get(f"https://www.serebii.net{link}", headers=headers)
        response.raise_for_status()
        text = response.text

    except requests.exceptions.RequestException as e:
        print(f"Request error {e} occured")
        return None
    soup = BeautifulSoup(text, "lxml")
    # get moves
    moves = []
    moves_data = soup.find_all("td", class_="fooinfo")
    name = moves_data[1].text
    for data in moves_data:
        if data.find("a") != None:
            link = data.find("a").get("href")
            if link != None:
                if "attackdex" in link:
                    move_data = data.find("a").text
                    if "TM" in move_data:
                        None
                    else:
                        moves.append(move_data)
    return pd.DataFrame({"name": name, "moves": moves})

In [34]:
for i, link in enumerate(links.loc[:, "links"]):
    if i < 1:
        pokemon = scrap_pokemon_moves(link, headers)
    else:
        pokemon = pd.concat([pokemon, scrap_pokemon_moves(link, headers)])
        

In [43]:
pokemon.index = range(0, 93345)

In [45]:
pokemon.to_csv("../pokemon_moves.csv")